<a href="https://colab.research.google.com/github/undefinedzack/stock-market-prediction-using-sentiment-analysis/blob/master/Copy_of_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data Manipulation

In [19]:
from google.colab import files 
uploaded = files.upload()

Saving tweet_sentiment.csv to tweet_sentiment.csv


In [31]:
import pandas as pd 
import io
import random

In [2]:
df = pd.read_csv(io.BytesIO(uploaded['stockerbot-export1.csv'])) 
senti = pd.read_csv(io.BytesIO(uploaded['tweet_sentiment.csv'])) 

In [18]:
df.head(10)

,id,text,timestamp,source,symbols,company_names,url,verified,cleaned_tweets,tokenized_text,stemmed_tokens
0,1.0197E+18,VIDEO: “I was in my office. I was minding my o...,Wed Jul 18 21:33:26 +0000 2018,GoldmanSachs,GS,The Goldman Sachs,https://twitter.com/i/web/status/1019696670777...,TRUE,video i was in my office i was minding my own ...,"[video, was, in, my, office, was, minding, my,...","[video, wa, in, my, offic, wa, mind, my, own, ..."
1,1.01971E+18,The price of lumber $LB_F is down 22% since hi...,Wed Jul 18 22:22:47 +0000 2018,StockTwits,M,Macy's,https://twitter.com/i/web/status/1019709091038...,TRUE,the price of lumber lb f is down since hitting...,"[the, price, of, lumber, lb, is, down, since, ...","[the, price, of, lumber, lb, is, down, sinc, h..."
2,1.01971E+18,Who says the American Dream is dead? https://t...,Wed Jul 18 22:32:01 +0000 2018,TheStreet,AIG,American,https://buff.ly/2L3kmc4,TRUE,who says the american dream is dead,"[who, says, the, american, dream, is, dead]","[who, sai, the, american, dream, is, dead]"
3,1.01972E+18,Barry Silbert is extremely optimistic on bitco...,Wed Jul 18 22:52:52 +0000 2018,MarketWatch,BTC,Bitcoin,https://twitter.com/i/web/status/1019716662587...,TRUE,barry silbert is extremely optimistic on bitco...,"[barry, silbert, is, extremely, optimistic, on...","[barri, silbert, is, extrem, optimist, on, bit..."
4,1.01972E+18,How satellites avoid attacks and space junk wh...,Wed Jul 18 23:00:01 +0000 2018,Forbes,ORCL,Oracle,http://on.forbes.com/6013DqDDU,TRUE,how satellites avoid attacks and space junk wh...,"[how, satellites, avoid, attacks, and, space, ...","[how, satellit, avoid, attack, and, space, jun..."
5,1.01972E+18,.@RealMoney's David Butler's favorite FANG sto...,Wed Jul 18 23:04:00 +0000 2018,jimcramer,FB-GOOGL-GOOG,Facebook*Alphabet*Alphabet,http://bit.ly/2NrYxje,TRUE,s david butler s favorite fang stock isn t rea...,"[david, butler, favorite, fang, stock, isn, re...","[david, butler, favorit, fang, stock, isn, rea..."
6,1.01972E+18,Don’t miss my convo with one of my favorite th...,Wed Jul 18 23:06:58 +0000 2018,ianbremmer,HRS,Harris,https://twitter.com/samharrisorg/status/101971...,TRUE,don t miss my convo with one of my favorite th...,"[don, miss, my, convo, with, one, of, my, favo...","[don, miss, my, convo, with, on, of, my, favor..."
7,1.01972E+18,U.S. intelligence documents on Nelson Mandela ...,Wed Jul 18 23:08:45 +0000 2018,Reuters,INTC-USB,Intel*U.S.,https://reut.rs/2O0ypNf,TRUE,u s intelligence documents on nelson mandela m...,"[intelligence, documents, on, nelson, mandela,...","[intellig, document, on, nelson, mandela, made..."
8,1.01972E+18,Senate wants emergency alerts to go out throug...,Wed Jul 18 23:09:00 +0000 2018,TechCrunch,NFLX,Netflix,https://tcrn.ch/2L8DsgT,TRUE,senate wants emergency alerts to go out throug...,"[senate, wants, emergency, alerts, to, go, out...","[senat, want, emerg, alert, to, go, out, throu..."
9,1.01972E+18,Hedge fund manager Marc Larsy says bitcoin $40...,Wed Jul 18 23:10:41 +0000 2018,MarketWatch,BTC,Bitcoin,https://on.mktw.net/2Ntr7k9,TRUE,hedge fund manager marc larsy says bitcoin k i...,"[hedge, fund, manager, marc, larsy, says, bitc...","[hedg, fund, manag, marc, larsi, sai, bitcoin,..."


In [4]:
df['text'][1]

"The price of lumber $LB_F is down 22% since hitting its YTD highs. The Macy's $M turnaround is still happening.… https://t.co/XnKsV4De39"

In [5]:
len(df['text'])

28440

In [6]:
##regular expressions for ids and links from text section
pat1= r'@[A-Za-z0-9]+'
pat2= r'https?://[A-Za-z0-9./]+'
#pat1|pat2
combined_pat=r'|'.join((pat1,pat2))
#no word should start with special symbol
pat3= r'[^a-zA-Z]'
combined_pat2=r'|'.join((combined_pat,pat3))

In [7]:
#demonstration
import re
re.sub(combined_pat2,' ','The price of lumber $LB_F is down 22% since hitting its YTD highs. The Macys $M turnaround is still happening.… https://t.co/XnKsV4De39')

'The price of lumber  LB F is down     since hitting its YTD highs  The Macys  M turnaround is still happening    '

In [8]:
# import nltk
# from nltk.corpus import stopwords
# from nltk.stem.porter import PorterStemmer
# nltk.download("popular")
# ps= PorterStemmer()
cleaned_tweets = []

for i in range(0,len(df['text'])):
    tweets = re.sub(combined_pat2,' ',df['text'][i])
    tweets=tweets.lower()
    tweets=tweets.split()
    # tweets = [ps.stem(word) for word in tweets if not word in set(stopwords.words('english'))]
    tweets = ' '.join(tweets)
    cleaned_tweets.append(tweets)
df['cleaned_tweets']=cleaned_tweets

In [9]:
df['cleaned_tweets'].head(10)

0    video i was in my office i was minding my own ...
1    the price of lumber lb f is down since hitting...
2                  who says the american dream is dead
3    barry silbert is extremely optimistic on bitco...
4    how satellites avoid attacks and space junk wh...
5    s david butler s favorite fang stock isn t rea...
6    don t miss my convo with one of my favorite th...
7    u s intelligence documents on nelson mandela m...
8    senate wants emergency alerts to go out throug...
9    hedge fund manager marc larsy says bitcoin k i...
Name: cleaned_tweets, dtype: object

# !!!Removal of Stop words!!!

In [ ]:
# from gensim.parsing.preprocessing import remove_stopwords

In [ ]:
# removed_stopwords_tweets = []

# for tweet in df['cleaned_tweets']:
#   temp = remove_stopwords(tweet)
#   removed_stopwords_tweets.append(temp)

# Tokenization

In [10]:
from gensim.utils import simple_preprocess

In [12]:
df['tokenized_text'] = [simple_preprocess(tweet, deacc=True) for tweet in df['cleaned_tweets']]

In [13]:
df['tokenized_text'].head(10)

0    [video, was, in, my, office, was, minding, my,...
1    [the, price, of, lumber, lb, is, down, since, ...
2          [who, says, the, american, dream, is, dead]
3    [barry, silbert, is, extremely, optimistic, on...
4    [how, satellites, avoid, attacks, and, space, ...
5    [david, butler, favorite, fang, stock, isn, re...
6    [don, miss, my, convo, with, one, of, my, favo...
7    [intelligence, documents, on, nelson, mandela,...
8    [senate, wants, emergency, alerts, to, go, out...
9    [hedge, fund, manager, marc, larsy, says, bitc...
Name: tokenized_text, dtype: object

# Stemming

In [14]:
from gensim.parsing.porter import PorterStemmer

In [15]:
PS = PorterStemmer()

df['stemmed_tokens'] = [ [PS.stem(word) for word in token] for token in df['tokenized_text']]

In [16]:
df['stemmed_tokens'].head(10)

0    [video, wa, in, my, offic, wa, mind, my, own, ...
1    [the, price, of, lumber, lb, is, down, sinc, h...
2           [who, sai, the, american, dream, is, dead]
3    [barri, silbert, is, extrem, optimist, on, bit...
4    [how, satellit, avoid, attack, and, space, jun...
5    [david, butler, favorit, fang, stock, isn, rea...
6    [don, miss, my, convo, with, on, of, my, favor...
7    [intellig, document, on, nelson, mandela, made...
8    [senat, want, emerg, alert, to, go, out, throu...
9    [hedg, fund, manag, marc, larsi, sai, bitcoin,...
Name: stemmed_tokens, dtype: object

# Splitting into Training and Testing set

In [17]:
from sklearn.model_selection import train_test_split

In [27]:
senti.head(5)

,cleaned_tweets,sentiment
0,video offic mind busi david solomon tell gs in...,0
1,price lumber lb f sinc hit ytd high maci turna...,0
2,say american dream dead,-1
3,barri silbert extrem optimist bitcoin predict ...,1
4,satellit avoid attack space junk circl earth paid,-1


In [30]:
df['text'].head(5)

0    VIDEO: “I was in my office. I was minding my o...
1    The price of lumber $LB_F is down 22% since hi...
2    Who says the American Dream is dead? https://t...
3    Barry Silbert is extremely optimistic on bitco...
4    How satellites avoid attacks and space junk wh...
Name: text, dtype: object

In [ ]:
random.seed(0)

In [33]:
x_train, x_test, y_train, y_test = train_test_split(
    df['stemmed_tokens'],
    senti['sentiment'],
    shuffle = True,
    test_size = 0.3,
    random_state = random.randint(1,100)
)

In [40]:
print(f'Training Samples :\n{y_train.value_counts()}')

Training Samples :
 0    12130
 1     5969
-1     1809
Name: sentiment, dtype: int64


In [41]:
print(f'Testing Samples :\n{y_test.value_counts()}')

Testing Samples :
 0    5200
 1    2543
-1     789
Name: sentiment, dtype: int64
